<a href="https://colab.research.google.com/github/Vitecnico/Projeto-Estudos/blob/main/Projeto_para_Estudos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PROJETO AQUECIMENTO

## Instalação das bibliotecas

In [ ]:
!pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importação das bibliotecas

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


In [ ]:
from google.cloud import storage
import os

In [ ]:
import pandas as pd
import pandera as pa

In [ ]:
import pymongo
from pymongo import MongoClient


## Chave de segurança Google

In [ ]:
serviceAccount = '/content/aulas-bc26-laisa-510383ea5245.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

## Importação e extração de datasets

### Cloud Storage/Bucket

In [ ]:
client = storage.Client()
bucket = client.get_bucket('projetodupladinamica')
#REALIZANDO O ACESSO A BUCKET


blob = bucket.blob('CAT.202201.orig.csv')
blob.upload_from_filename('/content/CAT.202201.orig.csv')

blob = bucket.blob('CAT.202204.orig.csv')
blob.upload_from_filename('/content/CAT.202204.orig.csv')

blob = bucket.blob('CAT.202210.orig.csv')
blob.upload_from_filename('/content/CAT.202210.orig.csv')

#ENVIANDO OS ARQUIVOS PARA A BUCKET

In [ ]:
path1 = 'gs://projetodupladinamica/CAT.202201.orig.csv'
path2 = 'gs://projetodupladinamica/CAT.202204.orig.csv'
path3 = 'gs://projetodupladinamica/CAT.202210.orig.csv'

#CAMINHOS DOS ARQUIVOS

### DataFrames

In [ ]:
df1 = pd.read_csv(path1, sep=';', encoding = 'ISO-8859-1')

#FAZENDO CRIANÇÃO DO DATAFRAME

In [ ]:
df1.head(4)

#VISUALIZAÇÃO DO DATAFRAME

,Agente Causador Acidente,Data Acidente,CBO,CID-10,CNAE2.0 Empregador,CNAE2.0 Empregador.1,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,...,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Data Acidente.1,Data Despacho Benefício,Data Acidente.2,Data Nascimento,Data Emissão CAT,CNPJ/CEI Empregador
0,{ñ class},2022/01,515105-Agente Comunitário de Saúde,B34.2 Infecc p/Coronavirus Ne,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,...,Feminino,Doença,Maranhão,São Paulo,2022/01,0000/00,20/01/2022,02/08/1970,01/03/2022,57.571.275.002.570
1,"Motocicleta, Motoneta",2022/02,519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,...,Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162
2,"Motocicleta, Motoneta",2022/02,519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,...,Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162
3,"Aprision. Em, Sobre ou Entre Dois ou Mai",2022/02,514205-Coletor Lixo,S60.0 Contusao de Dedos s/Lesao da Unha,3811,Coleta de Residuos Nao-Perigosos,Empregador,Pa,Empregado,Não,...,Masculino,Típico,{ñ class},Goiás,2022/02,0000/00,24/02/2022,19/09/1986,01/03/2022,00.000.000.000.000


In [ ]:
df2 = pd.read_csv(path2, sep=';', encoding = 'ISO-8859-1')

In [ ]:
df2.head(4)

,Agente Causador Acidente,Data Acidente,CBO,CID-10,CNAE2.0 Empregador,CNAE2.0 Empregador.1,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,...,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Data Acidente.1,Data Despacho Benefício,Data Acidente.2,Data Nascimento,Data Emissão CAT,CNPJ/CEI Empregador
0,{ñ class},2022/05,322205-Tec. de Enfermagem,Y28.2 Escolas Outr Instit Areas Admin Publica,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,...,Masculino,Típico,Maranhão,São Paulo,2022/05,0000/00,26/05/2022,13/11/1989,01/06/2022,57.571.275.002.570
1,Rua e Estrada - Superficie Utilizada para Sus,2022/05,992225-Auxiliar Geral de Conservação de Vias,S93.4 Entorse e Distensao do Tornozelo,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,...,Feminino,Trajeto,Maranhão,São Paulo,2022/05,0000/00,26/05/2022,19/04/1990,01/06/2022,57.571.275.002.570
2,{ñ class},2022/05,{ñ class},B34.2 Infecc p/Coronavirus Ne,8513,Ensino Fundamental,Empregador,Pa,Empregado,Não,...,Feminino,Doença,Maranhão,São Paulo,2022/05,0000/00,29/05/2022,13/09/1960,01/06/2022,47.715.347.000.184
3,"Revestimento Ceramico (Azulejo, Mosaico, Etc.",2022/05,783225-Ajudante de Motorista,S90.3 Contusao de Outr Partes e Partes Ne Do,4744,"Comercio Varejista de Ferragens, Madeira e Ma",Empregador,Pa,Empregado,Não,...,Masculino,Típico,Maranhão,São Paulo,2022/05,0000/00,31/05/2022,17/07/1994,01/06/2022,38.300.481.000.123


In [ ]:
df3 = pd.read_csv(path3, sep=';', encoding = 'ISO-8859-1')

In [ ]:
df3.head(4)

,Agente Causador Acidente,Data Acidente,CBO,CID-10,CNAE2.0 Empregador,CNAE2.0 Empregador.1,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,...,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Data Acidente.1,Data Despacho Benefício,Data Acidente.2,Data Nascimento,Data Emissão CAT,CNPJ/CEI Empregador
0,"Faca, Facao- Ferramenta Manual sem Forca Motr",2022/10,848515-Desossador,S51.0 Ferim do Cotovelo,1011,"Abate de Reses, Exceto Suinos",Empregador,Pa,Empregado,Não,...,Masculino,Típico,{ñ class},Goiás,2022/10,0000/00,05/10/2022,05/11/1988,06/10/2022,00.000.000.000.000
1,"Aprision. Em, Sob ou Entre, Nic",2022/10,324115-Tec. em Radiologia e Imagenologia,S60.0 Contusao de Dedos s/Lesao da Unha,8610,Atividades de Atendimento Hospitalar,Empregador,Pa,Empregado,Não,...,Masculino,Típico,Maranhão,São Paulo,2022/10,0000/00,04/10/2022,02/02/1989,06/10/2022,00.000.000.000.000
2,"Andaime, Plataforma - Edificio ou Estrutura",2022/10,715210-Pedreiro,S62.8 Frat de Outr Partes e de Partes Ne Punh,4120,Construcao de Edificios,Empregador,Pa,Empregado,Não,...,Masculino,Típico,{ñ class},Santa Catarina,2022/10,0000/00,05/10/2022,07/02/1981,06/10/2022,00.000.000.000.000
3,Veiculo Rodoviario Motorizado,2022/10,784205-Alimentador de Linha de Prod.,T02.2 Frat Envolv Regioes Mult de Um Membro S,4211,Construcao de Rodovias e Ferrovias,Empregador,Pa,Empregado,Sim,...,Masculino,Típico,Rondônia,Minas Gerais,2022/10,0000/00,05/10/2022,27/09/1958,06/10/2022,00.000.000.000.000


In [ ]:
df4 = pd.concat([df1, df2, df3])

#AGRUPANDO OS DATAFRAMES EM UM SÓ

In [ ]:
df4

,Agente Causador Acidente,Data Acidente,CBO,CID-10,CNAE2.0 Empregador,CNAE2.0 Empregador.1,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,...,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Data Acidente.1,Data Despacho Benefício,Data Acidente.2,Data Nascimento,Data Emissão CAT,CNPJ/CEI Empregador
0,{ñ class},2022/01,515105-Agente Comunitário de Saúde,B34.2 Infecc p/Coronavirus Ne,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,...,Feminino,Doença,Maranhão,São Paulo,2022/01,0000/00,20/01/2022,02/08/1970,01/03/2022,57.571.275.002.570
1,"Motocicleta, Motoneta",2022/02,519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,...,Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162
2,"Motocicleta, Motoneta",2022/02,519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,...,Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162
3,"Aprision. Em, Sobre ou Entre Dois ou Mai",2022/02,514205-Coletor Lixo,S60.0 Contusao de Dedos s/Lesao da Unha,3811,Coleta de Residuos Nao-Perigosos,Empregador,Pa,Empregado,Não,...,Masculino,Típico,{ñ class},Goiás,2022/02,0000/00,24/02/2022,19/09/1986,01/03/2022,00.000.000.000.000
4,Impacto de Pes. Contra Objeto em Movimento,2022/01,{ñ class},S90.0 Contusao do Tornozelo,4639,Comercio Atacadista de Produtos Alimenticios,Empregador,Pa,Empregado,Não,...,Masculino,Típico,Maranhão,São Paulo,2022/01,0000/00,28/01/2022,25/07/1995,01/03/2022,00.000.000.000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15654,Piso de Edificio - Superficie Utilizada Para,2022/09,{ñ class},S62 Frat ao Nivel do Punho e da Mao,2330,"Fabricacao de Artefatos de Concreto, Cimento,",{ñ class},Auxílio Doenca por Acidente do Trabalho,Empregado,Não,...,Masculino,Típico,Roraima,Paraná,2022/09,2022/10,29/09/2022,18/09/1993,10/10/2022,21.161.901.000.135
15655,Escada Permanente Cujos Degraus Permitem Apoi,2022/10,232130-Prof. Física no Ensino Médio,S82 Frat da Perna Incl Tornozelo,8520,Ensino Medio,Autoridade Pública,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,...,Feminino,Típico,Roraima,Paraná,2022/10,2022/11,06/10/2022,24/11/1986,07/10/2022,76.416.965.000.121
15656,"Andaime, Plataforma - Edificio ou Estrutura",2022/09,517330-Vigilante,S82.7 Frat Mult da Perna,8112,Condominios Prediais,Segurado/Dependente,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,...,Masculino,Típico,{ñ class},Santa Catarina,2022/09,2022/10,20/09/2022,02/03/1986,10/10/2022,32.994.401.000.109
15657,"Martelo, Malho, Marreta- Ferramenta Manual Se",2022/10,724315-Soldador,S62.6 Frat de Outr Dedos,7112,Servicos de Engenharia,Empregador,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,...,Masculino,Típico,Roraima,Paraná,2022/10,2022/10,05/10/2022,08/04/1984,06/10/2022,34.197.591.000.150


In [ ]:
dfback = df4.copy() 

#BACKUP DO DATAFRAME

#### Pré-análise

In [ ]:
df1.shape

#CONFERÊNCIA DO TOTAL DE LINHAS E COLUNAS

(152140, 24)

In [ ]:
df2.shape

(89602, 24)

In [ ]:
df3.shape

(15659, 24)

In [ ]:
df4.shape

(257401, 24)

## mongo

In [ ]:
uri = "mongodb+srv://laisa-soulcode.dtn7rgn.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,tls=True, tlsCertificateKeyFile='/content/X509-cert-729914187552401024.pem')


In [ ]:
db = client['INSS']
colecao1orig = db['CAT_ORIG1']



In [ ]:
db2 = client['INSS2']
colecao2orig = db2['CAT_ORIG2']


In [ ]:
db3 = client['INSS']
colecao3orig = db3['CAT_ORIG3']

In [ ]:
df_mg1_pt1 = df1.loc[0:40000]

In [ ]:
df_mg1_pt2 = df1.loc[40001:80000]

In [ ]:
df_mg1_pt3 = df1.loc[80001:120000]

In [ ]:
df_mg1_pt4 = df1.loc[120001:152140]

In [ ]:
df_mg1_dict1 = df_mg1_pt1.to_dict(orient = 'records')

In [ ]:
df_mg1_dict2 = df_mg1_pt2.to_dict(orient = 'records')

In [ ]:
df_mg1_dict3 = df_mg1_pt3.to_dict(orient = 'records')

In [ ]:
df_mg1_dict4 = df_mg1_pt4.to_dict(orient = 'records')

In [ ]:
df_mg2_pt1 = df2.loc[0:30000]

In [ ]:
df_mg2_pt2 = df2.loc[30001:50000]

In [ ]:
df_mg2_pt3 = df2.loc[50001:70000]

In [ ]:
df_mg2_pt4 = df2.loc[70001:89602]

In [ ]:
df_mg2_dict1 = df_mg2_pt1.to_dict(orient = 'records')

In [ ]:
df_mg2_dict2 = df_mg2_pt2.to_dict(orient = 'records')

In [ ]:
df_mg2_dict3 = df_mg2_pt3.to_dict(orient = 'records')

In [ ]:
df_mg2_dict4 = df_mg2_pt4.to_dict(orient = 'records')

In [ ]:
df_mg3_total = df3.loc[0:15659]

In [ ]:
df_mg3 = df_mg3_total.to_dict(orient = 'records')

In [ ]:
db.colecao1orig.insert_many(df_mg1_dict1)

In [ ]:
db.colecao1orig.insert_many(df_mg1_dict2)

In [ ]:
db.colecao1orig.insert_many(df_mg1_dict3)

In [ ]:
db.colecao1orig.insert_many(df_mg1_dict4)

In [ ]:
db2.colecao2orig.insert_many(df_mg2_dict1)

In [ ]:
db2.colecao2orig.insert_many(df_mg2_dict2)

In [ ]:
db2.colecao2orig.insert_many(df_mg2_dict3)

In [ ]:
db.colecao3orig.insert_many(df_mg2_dict1)

In [ ]:
#TENTAMOS FAAZER O ENVIO, PORÉM DEU ERRO NO MEIO DO PROCESSO :(

## Análise e tratamento

In [ ]:
df4.dtypes

#VERIFICAÇÃO DOS TIPOS DE DADOS

Agente  Causador  Acidente     object
Data Acidente                  object
CBO                            object
CID-10                         object
CNAE2.0 Empregador              int64
CNAE2.0 Empregador.1           object
Emitente CAT                   object
Espécie do benefício           object
Filiação Segurado              object
Indica Óbito Acidente          object
Munic Empr                     object
Natureza da Lesão              object
Origem de Cadastramento CAT    object
Parte Corpo Atingida           object
Sexo                           object
Tipo do Acidente               object
UF  Munic.  Acidente           object
UF Munic. Empregador           object
Data Acidente.1                object
Data Despacho Benefício        object
Data Acidente.2                object
Data Nascimento                object
Data Emissão CAT               object
CNPJ/CEI Empregador            object
dtype: object

In [ ]:
df4.rename(columns={'Agente  Causador  Acidente':'Agente_causador_acidente',
                   'Data Acidente':'Data_acidente',
                   'CBO':'CBO',
                   'CID-10':'CID',
                   'CNAE2.0 Empregador':'CNAE_empregador_num',
                   'CNAE2.0 Empregador.1':'CNAE2_empregador',
                   'Emitente CAT':'Emitente_cat',
                   'Espécie do benefício':'Especie_beneficio',
                   'Filiação Segurado':'Filiacao_segurado',
                   'Indica Óbito Acidente':'Obito_acidente',
                   'Munic Empr':'Municipio_empregador',
                   'Natureza da Lesão':'Natureza_lesao',
                   'Origem de Cadastramento CAT':'Origem_cadastro_cat',
                   'Parte Corpo Atingida':'Parte_corpo_atingida',
                   'Sexo':'Sexo',
                   'Tipo do Acidente':'Tipo_acidente',
                   'UF  Munic.  Acidente':'Uf_municipio_acidente',
                   'UF Munic. Empregador':'Uf_municipio_empregador',
                   'Data Acidente.1':'Data_acidente1',
                   'Data Despacho Benefício':'Data_despacho_beneficio', 
                   'Data Acidente.2':'Data_acidente2',
                   'Data Nascimento':'Data_nascimento',
                   'Data Emissão CAT':'Data_emissao_cat',
                   'CNPJ/CEI Empregador':'CNPJ_CEI_empregador'

},inplace=True)

#RENOMEANDO COLUNAS

In [ ]:
df4.columns.str.strip()

#RETIRANDO OS ESPAÇAMENTOS EM BRANCO

Index(['Agente_causador_acidente', 'Data_acidente', 'CBO', 'CID',
       'CNAE_empregador_num', 'CNAE2_empregador', 'Emitente_cat',
       'Especie_beneficio', 'Filiacao_segurado', 'Obito_acidente',
       'Municipio_empregador', 'Natureza_lesao', 'Origem_cadastro_cat',
       'Parte_corpo_atingida', 'Sexo', 'Tipo_acidente',
       'Uf_municipio_acidente', 'Uf_municipio_empregador', 'Data_acidente1',
       'Data_despacho_beneficio', 'Data_acidente2', 'Data_nascimento',
       'Data_emissao_cat', 'CNPJ_CEI_empregador'],
      dtype='object')

In [ ]:
df4.replace(['{','}','\(', '\)', '`', '´', '~'],'',regex=True,inplace=True)

#RETIRANDO CARACTER ESPECIAL

In [ ]:
df4

,Agente_causador_acidente,Data_acidente,CBO,CID,CNAE_empregador_num,CNAE2_empregador,Emitente_cat,Especie_beneficio,Filiacao_segurado,Obito_acidente,...,Sexo,Tipo_acidente,Uf_municipio_acidente,Uf_municipio_empregador,Data_acidente1,Data_despacho_beneficio,Data_acidente2,Data_nascimento,Data_emissao_cat,CNPJ_CEI_empregador
0,ñ class,2022/01,515105-Agente Comunitário de Saúde,B34.2 Infecc p/Coronavirus Ne,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,...,Feminino,Doença,Maranhão,São Paulo,2022/01,0000/00,20/01/2022,02/08/1970,01/03/2022,57.571.275.002.570
1,"Motocicleta, Motoneta",2022/02,519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,...,Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162
2,"Motocicleta, Motoneta",2022/02,519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,...,Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162
3,"Aprision. Em, Sobre ou Entre Dois ou Mai",2022/02,514205-Coletor Lixo,S60.0 Contusao de Dedos s/Lesao da Unha,3811,Coleta de Residuos Nao-Perigosos,Empregador,Pa,Empregado,Não,...,Masculino,Típico,ñ class,Goiás,2022/02,0000/00,24/02/2022,19/09/1986,01/03/2022,00.000.000.000.000
4,Impacto de Pes. Contra Objeto em Movimento,2022/01,ñ class,S90.0 Contusao do Tornozelo,4639,Comercio Atacadista de Produtos Alimenticios,Empregador,Pa,Empregado,Não,...,Masculino,Típico,Maranhão,São Paulo,2022/01,0000/00,28/01/2022,25/07/1995,01/03/2022,00.000.000.000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15654,Piso de Edificio - Superficie Utilizada Para,2022/09,ñ class,S62 Frat ao Nivel do Punho e da Mao,2330,"Fabricacao de Artefatos de Concreto, Cimento,",ñ class,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,...,Masculino,Típico,Roraima,Paraná,2022/09,2022/10,29/09/2022,18/09/1993,10/10/2022,21.161.901.000.135
15655,Escada Permanente Cujos Degraus Permitem Apoi,2022/10,232130-Prof. Física no Ensino Médio,S82 Frat da Perna Incl Tornozelo,8520,Ensino Medio,Autoridade Pública,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,...,Feminino,Típico,Roraima,Paraná,2022/10,2022/11,06/10/2022,24/11/1986,07/10/2022,76.416.965.000.121
15656,"Andaime, Plataforma - Edificio ou Estrutura",2022/09,517330-Vigilante,S82.7 Frat Mult da Perna,8112,Condominios Prediais,Segurado/Dependente,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,...,Masculino,Típico,ñ class,Santa Catarina,2022/09,2022/10,20/09/2022,02/03/1986,10/10/2022,32.994.401.000.109
15657,"Martelo, Malho, Marreta- Ferramenta Manual Se",2022/10,724315-Soldador,S62.6 Frat de Outr Dedos,7112,Servicos de Engenharia,Empregador,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,...,Masculino,Típico,Roraima,Paraná,2022/10,2022/10,05/10/2022,08/04/1984,06/10/2022,34.197.591.000.150


In [ ]:
df4.isnull().sum()

#VERIFICANDO SE HÁ DADOS AUSENTES

Agente_causador_acidente    0
Data_acidente               0
CBO                         0
CID                         0
CNAE_empregador_num         0
CNAE2_empregador            0
Emitente_cat                0
Especie_beneficio           0
Filiacao_segurado           0
Obito_acidente              0
Municipio_empregador        0
Natureza_lesao              0
Origem_cadastro_cat         0
Parte_corpo_atingida        0
Sexo                        0
Tipo_acidente               0
Uf_municipio_acidente       0
Uf_municipio_empregador     0
Data_acidente1              0
Data_despacho_beneficio     0
Data_acidente2              0
Data_nascimento             0
Data_emissao_cat            0
CNPJ_CEI_empregador         0
dtype: int64

In [ ]:
df4.count()

#CONSULTANDO OS DADOS VÁLIDOS

Agente_causador_acidente    257401
Data_acidente               257401
CBO                         257401
CID                         257401
CNAE_empregador_num         257401
CNAE2_empregador            257401
Emitente_cat                257401
Especie_beneficio           257401
Filiacao_segurado           257401
Obito_acidente              257401
Municipio_empregador        257401
Natureza_lesao              257401
Origem_cadastro_cat         257401
Parte_corpo_atingida        257401
Sexo                        257401
Tipo_acidente               257401
Uf_municipio_acidente       257401
Uf_municipio_empregador     257401
Data_acidente1              257401
Data_despacho_beneficio     257401
Data_acidente2              257401
Data_nascimento             257401
Data_emissao_cat            257401
CNPJ_CEI_empregador         257401
dtype: int64

#### *VERIFICANDO SE OS DADOS SÃO ÚNICOS E SE HÁ ALGUM DADO INCONSISTENTE*

In [ ]:
pd.unique(df4['Agente_causador_acidente'])

#VERIFICANDO SE OS DADOS E SE SÃO ÚNICOS

array(['ñ class                                    ',
       'Motocicleta, Motoneta                        ',
       'Aprision. Em, Sobre ou Entre Dois ou Mai     ',
       'Impacto de Pes. Contra Objeto em Movimento   ',
       'Temperatura Muito Alta, Contato com Objeto Ou',
       'Piso de Veiculo - Superficie Utilizada para S',
       'Frasco, Garrafa - Embalagem, Recipiente, Vazi',
       'Piso de Andaime e Plataforma Desmontavel - Su',
       'Produto Biologico Soro, Toxina, Antitoxina, ',
       'Passarela ou Plataforma Permanentes - Superfi',
       'Esmeril- Ferramenta Portatil com Forca Motriz',
       'Caixa, Engradado, Caixote - Embalagem, Recipi',
       'Atrito ou Abrasao por Manusear Objeto        ',
       'Talha - Equip. de Guindar                    ',
       'Rampa - Superficie Utilizada para Sustentar P',
       'Chao - Superficie Utilizada para Sustentar Pe',
       'Impacto de Pes. Contra Objeto Parado         ',
       'Queda de Pes. com Dif. de Nivel de Esc     

In [ ]:
pd.unique(df4['Data_acidente'])

array(['2022/01', '2022/02', '2022/03', '2022/05', '2022/06', '2022/04',
       '2022/10', '2022/09', '2022/08', '2022/07'], dtype=object)

In [ ]:
pd.unique(df4['CBO'])

array(['515105-Agente Comunitário de Saúde           ',
       '519110-Motociclista no Transporte de Document',
       '514205-Coletor Lixo                          ', ...,
       '214325-Engenheiro Eletrônico de Manut.       ',
       '142520-Ger. Projetos de Tecnologia da Informa',
       '351310-Tec. em Administração de Comércio Exte'], dtype=object)

In [ ]:
pd.unique(df4['CID'])

array(['B34.2 Infecc p/Coronavirus Ne                ',
       'S52.5 Frat da Extremidade Distal do Radio    ',
       'S60.0 Contusao de Dedos s/Lesao da Unha      ', ...,
       'L02.0 Abscesso Cutaneo Furunculo e Antraz Fac',
       'R46.2 Comport Estranho e Inexplicavel        ',
       'S74.1 Traum do Nervo Femural Nivel Quadril E '], dtype=object)

In [ ]:
pd.unique(df4['Emitente_cat'])


array(['Empregador         ', 'Segurado/Dependente',
       'Médico             ', 'ñ class          ', 'Autoridade Pública ',
       'Sindicato          '], dtype=object)

In [ ]:
pd.unique(df4['Especie_beneficio'])

array(['Pa                                           ',
       'Auxílio Doenca por Acidente do Trabalho      ',
       'Auxílio Acidente                             ',
       'Aposent. Invalidez Acidente Trabalho         ',
       'Pensão por Morte Acidente do Trabalho        '], dtype=object)

In [ ]:
pd.unique(df4['Obito_acidente'])

array(['Não', 'Sim', 'ñ '], dtype=object)

In [ ]:
pd.unique(df4['Municipio_empregador'])

array(['354780-Santo André-Sp                    ',
       '355030-São Paulo                         ',
       '520110-Anápolis                          ', ...,
       '410900-Guapirama                         ',
       '293220-Ubaitaba                          ',
       '420050-Águas de Chapecó                  '], dtype=object)

In [ ]:
pd.unique(df4['Sexo'])

array(['Feminino     ', 'Masculino    ', 'Não Informado', 'Indeterminado'],
      dtype=object)

In [ ]:
pd.unique(df4['Tipo_acidente'])

array(['Doença  ', 'Trajeto ', 'Típico  ', 'Ignorado'], dtype=object)

In [ ]:
pd.unique(df4['Uf_municipio_acidente'])

array(['Maranhão        ', 'ñ class       ', 'Pará            ',
       'Roraima         ', 'Zerado          ', 'Ceará           ',
       'Rondônia        ', 'Amazonas        ', 'Tocantins       ',
       'Pernambuco      ', 'Acre            ', 'Sergipe         ',
       'Rio Grande Norte', 'Amapá           ', 'Paraíba         ',
       'Alagoas         ', 'Piauí           '], dtype=object)

In [ ]:
pd.unique(df4['Uf_municipio_empregador'])

array(['São Paulo          ', 'Goiás              ',
       'Bahia              ', 'Pernambuco         ',
       'Paraná             ', 'Ceará              ',
       'Santa Catarina     ', 'Rio Grande do Sul  ',
       'Zerado             ', 'Mato Grosso do Sul ',
       'Distrito Federal   ', 'Minas Gerais       ',
       'Alagoas            ', 'Rio Grande do Norte',
       'Paraíba            ', 'Mato Grosso        ',
       'Rio de Janeiro     ', 'Rondônia           ',
       'Pará               ', 'Tocantins          ',
       'Acre               ', 'Piauí              ',
       'Maranhão           ', 'Espírito Santo     ',
       'Amapá              ', 'Amazonas           ',
       'Roraima            ', 'Sergipe            '], dtype=object)

In [ ]:
pd.unique(df4['Data_acidente1'])

array(['2022/01', '2022/02', '2022/03', '2022/05', '2022/06', '2022/04',
       '2022/10', '2022/09', '2022/08', '2022/07'], dtype=object)

In [ ]:
pd.unique(df4['Data_despacho_beneficio'])

array(['0000/00', '2022/01', '2022/03', '2022/02', '2022/04', '2022/07',
       '2022/06', '2022/05', '2022/11', '2022/10'], dtype=object)

In [ ]:
pd.unique(df4['Data_acidente2'])

array(['20/01/2022', '25/02/2022', '24/02/2022', '28/01/2022',
       '28/02/2022', '18/01/2022', '01/03/2022', '23/02/2022',
       '27/02/2022', '26/02/2022', '24/01/2022', '15/02/2022',
       '16/02/2022', '12/01/2022', '22/02/2022', '14/02/2022',
       '09/02/2022', '11/02/2022', '18/02/2022', '10/02/2022',
       '02/02/2022', '03/02/2022', '07/02/2022', '17/02/2022',
       '04/02/2022', '21/02/2022', '06/02/2022', '03/01/2022',
       '08/02/2022', '20/02/2022', '05/02/2022', '19/01/2022',
       '15/01/2022', '19/02/2022', '13/02/2022', '17/01/2022',
       '12/02/2022', '31/01/2022', '02/01/2022', '01/02/2022',
       '29/01/2022', '11/01/2022', '05/01/2022', '27/01/2022',
       '25/01/2022', '16/01/2022', '21/01/2022', '26/01/2022',
       '13/01/2022', '14/01/2022', '22/01/2022', '07/01/2022',
       '04/01/2022', '06/01/2022', '02/03/2022', '30/01/2022',
       '08/01/2022', '10/01/2022', '23/01/2022', '03/03/2022',
       '09/01/2022', '04/03/2022', '05/03/2022', '06/03

In [ ]:
pd.unique(df4['Data_nascimento'])

array(['02/08/1970', '16/12/1990', '19/09/1986', ..., '04/03/1954',
       '14/01/1962', '29/06/1952'], dtype=object)

In [ ]:
pd.unique(df4['Data_emissao_cat'])

array(['01/03/2022', '02/03/2022', '03/03/2022', '04/03/2022',
       '05/03/2022', '06/03/2022', '07/03/2022', '08/03/2022',
       '09/03/2022', '10/03/2022', '11/03/2022', '12/03/2022',
       '13/03/2022', '14/03/2022', '15/03/2022', '16/03/2022',
       '17/03/2022', '18/03/2022', '19/03/2022', '20/03/2022',
       '21/03/2022', '22/03/2022', '23/03/2022', '24/03/2022',
       '25/03/2022', '26/03/2022', '27/03/2022', '28/03/2022',
       '29/03/2022', '30/03/2022', '31/03/2022', '00/00/0000',
       '01/06/2022', '02/06/2022', '03/06/2022', '04/06/2022',
       '05/06/2022', '06/06/2022', '07/06/2022', '08/06/2022',
       '09/06/2022', '10/06/2022', '11/06/2022', '12/06/2022',
       '13/06/2022', '14/06/2022', '15/06/2022', '16/06/2022',
       '17/06/2022', '18/06/2022', '19/06/2022', '20/06/2022',
       '21/06/2022', '22/06/2022', '23/06/2022', '24/06/2022',
       '25/06/2022', '26/06/2022', '27/06/2022', '28/06/2022',
       '29/06/2022', '30/06/2022', '06/10/2022', '07/10

In [ ]:
pd.unique(df4['CNPJ_CEI_empregador'])

array(['57.571.275.002.570', '19.578.086.000.162', '00.000.000.000.000',
       ..., '32.994.401.000.109', '34.197.591.000.150',
       '54.614.813.000.109'], dtype=object)

In [ ]:
df4.groupby(['CNPJ_CEI_empregador']).size().head(40)

CNPJ_CEI_empregador
00.000.000.000.000    221805
00.000.000.000.191        32
00.000.000.014.737         2
00.000.000.116.491         1
00.000.000.118.516         2
00.000.000.143.626         4
00.000.000.297.003         2
00.000.063.074.770         1
00.000.078.197.066         2
00.000.138.000.271         1
00.000.138.000.433         1
00.000.189.490.888         1
00.000.208.000.100         1
00.000.209.708.700         3
00.000.322.989.604         1
00.000.352.418.109         1
00.000.393.539.903         2
00.000.400.092.620         1
00.000.403.842.760         1
00.000.436.915.014         2
00.000.438.138.600         2
00.000.463.809.815         2
00.000.516.171.615         1
00.000.580.357.619         2
00.000.681.889.608         1
00.000.750.581.972         1
00.000.773.664.416         1
00.000.816.267.030         1
00.000.819.284.750         1
00.001.017.373.604         2
00.001.114.412.856         1
00.001.172.000.180         2
00.001.176.764.896         1
00.001.203.905.157     

`Inconsistencias encontradas: ñ class, ñ e CNPJ's e datas zeradas`

In [ ]:
df4[['Data_acidente','Data_acidente1','Data_acidente2']]

# VISUALIZAR AS COLUNAS PARA VERIFICAR SE OS DADOS SÃO SEMELHANTES

,Data_acidente,Data_acidente1,Data_acidente2
0,2022/01,2022/01,20/01/2022
1,2022/02,2022/02,25/02/2022
2,2022/02,2022/02,25/02/2022
3,2022/02,2022/02,24/02/2022
4,2022/01,2022/01,28/01/2022
...,...,...,...
15654,2022/09,2022/09,29/09/2022
15655,2022/10,2022/10,06/10/2022
15656,2022/09,2022/09,20/09/2022
15657,2022/10,2022/10,05/10/2022


# *TRATANDO AS INCONSISTÊNCIAS E OS DADOS*

In [ ]:
df4.replace({'ñ class                                    ':'Nao_classificado'},inplace=True)
df4.replace({'ñ class          ':'Nao_classificado'},inplace=True)
df4.replace({'ñ class       ':'Nao_classificado'},inplace=True)

df4.replace({'ñ class          ':'Nao_classificado'},inplace=True)
df4.replace({'ñ class       ':'Nao_classificado'},inplace=True)
df4.replace({'ñ ':'Não'},inplace=True)
df4.replace({'00.000.000.000.000':'Nao_informado'},inplace=True)
df4.replace({'Zerado             ':'Nao_classificado'},inplace=True)
df4.replace({'Zerado          ':'Nao_classificado'},inplace=True)

#RENOMEANDO OS DADOS INCONSISTENTES PARA 'Nao_classificado' E OS CNPJS PARA 'Nao_informado' PARAA QUE NÃO APRESENTE, EM ALGUMA ANÁLISE FUTURA, ALGUM DUPLO ENTENDIMENTO PI ANÁLISE INCORRETA, DEIXANDO O DADO MAIS CLARO.

In [ ]:
df4.drop(['Data_acidente','Data_acidente1'],axis=1,inplace=True)

#DROP REALIZADO POIS AS COLUNAS ESTAVAM COM INFORMAÇÕES REPETIDAS E INUTILIZÁVEIS, VISTO QUE HÁ UMA COLUNA COM OS DADOS MAIS COMPLETOS

In [ ]:
df4['Data_despacho_beneficio'] = pd.to_datetime(df4['Data_despacho_beneficio'],errors='coerce')
df4['Data_acidente2'] = pd.to_datetime(df4['Data_acidente2'],errors='coerce')
df4['Data_nascimento'] = pd.to_datetime(df4['Data_nascimento'],errors='coerce')
df4['Data_emissao_cat'] = pd.to_datetime(df4['Data_emissao_cat'],errors='coerce')

#TROCANDO O TIPO DAS COLUNAS QUE CONTÉM DATAS PARA DATETIME

In [ ]:
df4.dtypes

#VERIFICANDO A CONVERSÃO DAS DATAS

Agente_causador_acidente            object
CBO                                 object
CID                                 object
CNAE_empregador_num                  int64
CNAE2_empregador                    object
Emitente_cat                        object
Especie_beneficio                   object
Filiacao_segurado                   object
Obito_acidente                      object
Municipio_empregador                object
Natureza_lesao                      object
Origem_cadastro_cat                 object
Parte_corpo_atingida                object
Sexo                                object
Tipo_acidente                       object
Uf_municipio_acidente               object
Uf_municipio_empregador             object
Data_despacho_beneficio     datetime64[ns]
Data_acidente2              datetime64[ns]
Data_nascimento             datetime64[ns]
Data_emissao_cat            datetime64[ns]
CNPJ_CEI_empregador                 object
dtype: object

In [ ]:
df4

,Agente_causador_acidente,CBO,CID,CNAE_empregador_num,CNAE2_empregador,Emitente_cat,Especie_beneficio,Filiacao_segurado,Obito_acidente,Municipio_empregador,...,Parte_corpo_atingida,Sexo,Tipo_acidente,Uf_municipio_acidente,Uf_municipio_empregador,Data_despacho_beneficio,Data_acidente2,Data_nascimento,Data_emissao_cat,CNPJ_CEI_empregador
0,Nao_classificado,515105-Agente Comunitário de Saúde,B34.2 Infecc p/Coronavirus Ne,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,354780-Santo André-Sp,...,Aparelho Respiratorio,Feminino,Doença,Maranhão,São Paulo,NaT,2022-01-20,1970-02-08,2022-01-03,57.571.275.002.570
1,"Motocicleta, Motoneta",519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,355030-São Paulo,...,Braco Entre O Punho a O Ombro,Masculino,Trajeto,Maranhão,São Paulo,NaT,2022-02-25,1990-12-16,2022-01-03,19.578.086.000.162
2,"Motocicleta, Motoneta",519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,355030-São Paulo,...,Braco Entre O Punho a O Ombro,Masculino,Trajeto,Maranhão,São Paulo,NaT,2022-02-25,1990-12-16,2022-01-03,19.578.086.000.162
3,"Aprision. Em, Sobre ou Entre Dois ou Mai",514205-Coletor Lixo,S60.0 Contusao de Dedos s/Lesao da Unha,3811,Coleta de Residuos Nao-Perigosos,Empregador,Pa,Empregado,Não,520110-Anápolis,...,Punho,Masculino,Típico,Nao_classificado,Goiás,NaT,2022-02-24,1986-09-19,2022-01-03,Nao_informado
4,Impacto de Pes. Contra Objeto em Movimento,Nao_classificado,S90.0 Contusao do Tornozelo,4639,Comercio Atacadista de Produtos Alimenticios,Empregador,Pa,Empregado,Não,330100-Campos dos Goytacazes,...,Perna Entre O Tornozelo e a Pelvis,Masculino,Típico,Maranhão,São Paulo,NaT,2022-01-28,1995-07-25,2022-01-03,Nao_informado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15654,Piso de Edificio - Superficie Utilizada Para,Nao_classificado,S62 Frat ao Nivel do Punho e da Mao,2330,"Fabricacao de Artefatos de Concreto, Cimento,",Nao_classificado,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,410960-Guaratuba,...,Mao Exceto Punho ou Dedos,Masculino,Típico,Roraima,Paraná,2022-10-01,2022-09-29,1993-09-18,2022-10-10,21.161.901.000.135
15655,Escada Permanente Cujos Degraus Permitem Apoi,232130-Prof. Física no Ensino Médio,S82 Frat da Perna Incl Tornozelo,8520,Ensino Medio,Autoridade Pública,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,410690-Curitiba,...,"Perna Do Tornozelo, Exclusive, ao Joelho, Ex",Feminino,Típico,Roraima,Paraná,2022-11-01,2022-06-10,1986-11-24,2022-07-10,76.416.965.000.121
15656,"Andaime, Plataforma - Edificio ou Estrutura",517330-Vigilante,S82.7 Frat Mult da Perna,8112,Condominios Prediais,Segurado/Dependente,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,420460-Criciúma,...,"Membros Inferiores, Partes Multiplas Qualque",Masculino,Típico,Nao_classificado,Santa Catarina,2022-10-01,2022-09-20,1986-02-03,2022-10-10,32.994.401.000.109
15657,"Martelo, Malho, Marreta- Ferramenta Manual Se",724315-Soldador,S62.6 Frat de Outr Dedos,7112,Servicos de Engenharia,Empregador,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,411000-Iguaraçu,...,Dedo,Masculino,Típico,Roraima,Paraná,2022-10-01,2022-05-10,1984-08-04,2022-06-10,34.197.591.000.150


## Arquivo concatenado

In [ ]:
df4.to_csv('INSS2022.csv')

In [ ]:
blob = bucket.blob('INSS2022.csv')
blob.upload_from_filename('/content/INSS2022.csv')


# ENVIO TRATADO

In [ ]:
df4

,Agente_causador_acidente,CBO,CID,CNAE_empregador_num,CNAE2_empregador,Emitente_cat,Especie_beneficio,Filiacao_segurado,Obito_acidente,Municipio_empregador,...,Parte_corpo_atingida,Sexo,Tipo_acidente,Uf_municipio_acidente,Uf_municipio_empregador,Data_despacho_beneficio,Data_acidente2,Data_nascimento,Data_emissao_cat,CNPJ_CEI_empregador
0,Nao_classificado,515105-Agente Comunitário de Saúde,B34.2 Infecc p/Coronavirus Ne,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,354780-Santo André-Sp,...,Aparelho Respiratorio,Feminino,Doença,Maranhão,São Paulo,NaT,2022-01-20,1970-02-08,2022-01-03,57.571.275.002.570
1,"Motocicleta, Motoneta",519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,355030-São Paulo,...,Braco Entre O Punho a O Ombro,Masculino,Trajeto,Maranhão,São Paulo,NaT,2022-02-25,1990-12-16,2022-01-03,19.578.086.000.162
2,"Motocicleta, Motoneta",519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,355030-São Paulo,...,Braco Entre O Punho a O Ombro,Masculino,Trajeto,Maranhão,São Paulo,NaT,2022-02-25,1990-12-16,2022-01-03,19.578.086.000.162
3,"Aprision. Em, Sobre ou Entre Dois ou Mai",514205-Coletor Lixo,S60.0 Contusao de Dedos s/Lesao da Unha,3811,Coleta de Residuos Nao-Perigosos,Empregador,Pa,Empregado,Não,520110-Anápolis,...,Punho,Masculino,Típico,Nao_classificado,Goiás,NaT,2022-02-24,1986-09-19,2022-01-03,Nao_informado
4,Impacto de Pes. Contra Objeto em Movimento,Nao_classificado,S90.0 Contusao do Tornozelo,4639,Comercio Atacadista de Produtos Alimenticios,Empregador,Pa,Empregado,Não,330100-Campos dos Goytacazes,...,Perna Entre O Tornozelo e a Pelvis,Masculino,Típico,Maranhão,São Paulo,NaT,2022-01-28,1995-07-25,2022-01-03,Nao_informado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15654,Piso de Edificio - Superficie Utilizada Para,Nao_classificado,S62 Frat ao Nivel do Punho e da Mao,2330,"Fabricacao de Artefatos de Concreto, Cimento,",Nao_classificado,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,410960-Guaratuba,...,Mao Exceto Punho ou Dedos,Masculino,Típico,Roraima,Paraná,2022-10-01,2022-09-29,1993-09-18,2022-10-10,21.161.901.000.135
15655,Escada Permanente Cujos Degraus Permitem Apoi,232130-Prof. Física no Ensino Médio,S82 Frat da Perna Incl Tornozelo,8520,Ensino Medio,Autoridade Pública,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,410690-Curitiba,...,"Perna Do Tornozelo, Exclusive, ao Joelho, Ex",Feminino,Típico,Roraima,Paraná,2022-11-01,2022-06-10,1986-11-24,2022-07-10,76.416.965.000.121
15656,"Andaime, Plataforma - Edificio ou Estrutura",517330-Vigilante,S82.7 Frat Mult da Perna,8112,Condominios Prediais,Segurado/Dependente,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,420460-Criciúma,...,"Membros Inferiores, Partes Multiplas Qualque",Masculino,Típico,Nao_classificado,Santa Catarina,2022-10-01,2022-09-20,1986-02-03,2022-10-10,32.994.401.000.109
15657,"Martelo, Malho, Marreta- Ferramenta Manual Se",724315-Soldador,S62.6 Frat de Outr Dedos,7112,Servicos de Engenharia,Empregador,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,411000-Iguaraçu,...,Dedo,Masculino,Típico,Roraima,Paraná,2022-10-01,2022-05-10,1984-08-04,2022-06-10,34.197.591.000.150


In [ ]:
df4.to_csv('INSS.trat')

#CONVERTENDO PARA CSV PARA ENVIAR O ARQUIVO TRATADO PARA A BUCKET

In [ ]:
blob = bucket.blob('INSS.trat')
blob.upload_from_filename('/content/INSS.trat')

#ENVIANDO O ARQUIVO TRATADO PARA A BUCKET